In [ ]:
!pip install unsloth

In [ ]:
from unsloth import FastModel

max_seq_length = 4_000

model, tokenizer = FastModel.from_pretrained(
    model_name="unsloth/gemma-3-4b-it-unsloth-bnb-4bit",
    max_seq_length=max_seq_length,
    load_in_4bit=True,
)

In [ ]:
model = FastModel.get_peft_model(
    model,
    r=8,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj", ],
    lora_alpha=8,
)

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template="gemma3",
)

In [ ]:
from datasets import load_dataset

dataset = load_dataset("IRI2070/tax-quize-fine-tuning-toy-dataset")

In [ ]:
def convert_to_chatml(item):
    messages = []

    system_prompt = (
        "#USE_TAGGED_OUTPUT#\n"
        "شما یک دستیار هوش مصنوعی متخصص در پرسش‌های چهارگزینه‌ای مربوط به قانون مالیات ایران هستید. "
        "وظیفه‌ی شما این است که سؤال، گزینه‌ها و زمینه‌ی ارائه‌شده را با دقت بخوانید، سپس گزینه‌ی صحیح "
        "(A، B، C یا D) را انتخاب کرده و یک استدلال کوتاه اما روشن ارائه دهید.\n\n"
        "راهنما:\n"
        "- خروجی باید فقط شامل دو بخش باشد:\n"
        "   <thinking> توضیح کوتاه به فارسی </thinking>\n"
        "   <answer> یکی از A,B,C,D </answer>\n"
        "- استدلال باید توضیح دهد چرا گزینه‌ی انتخاب‌شده درست است و چرا سایر گزینه‌ها نادرست هستند.\n"
        "- لحن پاسخ باید حرفه‌ای، دقیق و معتبر باشد.\n"
        "- در پاسخ، سؤال یا گزینه‌ها را دوباره بازنویسی نکنید.\n"
    )

    messages.append({"role": "system", "content": system_prompt})

    user_content = f"سوال: {item['question']}\n\گزینه ها:\n"
    for key, val in item["options"].items():
        user_content += f"{key}: {val}\n"
    user_content += f"\nمتن زمینه: {item.get('context', '')}\n"

    messages.append({"role": "user", "content": user_content})

    assistant_content = f"<thinking>{item['reasoning']}</thinking><answer>{item['answer']}</answer>"

    messages.append({"role": "assistant", "content": assistant_content})

    return {
        "conversations": messages
    }


dataset = dataset['train'].map(
    convert_to_chatml
)

In [ ]:
def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False).removeprefix('<bos>') for
             convo in convos]
    return {"text": texts, }


dataset = dataset.map(formatting_prompts_func, batched=True)

In [ ]:
from trl import SFTTrainer, SFTConfig

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    eval_dataset=None,
    args=SFTConfig(
        dataset_text_field="text",
        per_device_train_batch_size=2,
        gradient_accumulation_steps=8,
        warmup_steps=5,
        num_train_epochs=3,
        learning_rate=2e-4,
        logging_steps=1,
        output_dir="outputs",
        report_to="none",
    ),
)

In [ ]:
from unsloth.chat_templates import train_on_responses_only

trainer = train_on_responses_only(
    trainer,
    instruction_part="<start_of_turn>user\n",
    response_part="<start_of_turn>model\n",
)

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 80 | Num Epochs = 3 | Total steps = 15
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 8 x 1) = 16
 "-____-"     Trainable parameters = 16,394,240 of 4,316,473,712 (0.38% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.414700


In [ ]:
model.push_to_hub("<user-name>/gemma-3-4b-it-unsloth-bnb-4bit-tax-quize-fine-tuned", token = "hf_XXXXXXXXXXXXXXXXXXXXX")
tokenizer.push_to_hub("<user-name>/gemma-3-4b-it-unsloth-bnb-4bit-tax-quize-fine-tuned", token = "hf_XXXXXXXXXXXXXXXXXXXXX")